In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
imageSize = [299, 299]

trainpath = r"/content/drive/MyDrive/Parkdataset/training";

testPath = r"/content/drive/MyDrive/Parkdataset/testing"

In [ ]:
inception = InceptionV3(input_shape=imageSize + [3], weights='imagenet',include_top=False)

In [ ]:
for layer in inception.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(inception.output)

In [ ]:
prediction = Dense(3, activation='softmax')(x)

In [ ]:
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_282 (Conv2D)            (None, 149, 149, 32  864         ['input_8[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_282 (Batch  (None, 149, 149, 32  96         ['conv2d_282[0][0]']             
 Normalization)                 )                                                           

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(trainpath,
                                                 target_size = (299, 299),
                                                 batch_size = 7,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(testPath,
                                            target_size = (299, 299),
                                            batch_size = 7,
                                            class_mode = 'categorical')

Found 102 images belonging to 3 classes.
Found 41 images belonging to 3 classes.


In [ ]:
training_set.class_indices

{'Error': 0, 'healthy': 1, 'parkinson': 2}

In [ ]:
len(training_set)

15

In [ ]:
len(test_set)

6

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set)//5,
  validation_steps=len(test_set)//6
)

<ipython-input-104-8491a40f731a>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/5
3/3 [==============================] - 32s 5s/step - loss: 1.3411e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/5
3/3 [==============================] - 8s 2s/step - loss: 5.3595 - accuracy: 0.8333 - val_loss: 1.1865e-04 - val_accuracy: 1.0000
Epoch 3/5
3/3 [==============================] - 7s 3s/step - loss: 5.3696 - accuracy: 0.8889 - val_loss: 6.2598 - val_accuracy: 0.8571
Epoch 4/5
3/3 [==============================] - 8s 3s/step - loss: 0.9262 - accuracy: 0.9524 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/5
3/3 [==============================] - 9s 3s/step - loss: 3.7557 - accuracy: 0.8889 - val_loss: 5.4124 - val_accuracy: 0.7143


In [ ]:
model.save('Inception-parkinson.h5')

In [ ]:
#import load_model class for loading h5 file
from tensorflow.keras.models import load_model
#import image class to process the images
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np

In [ ]:
model=load_model('Inception-parkinson.h5')

In [ ]:
img=image.load_img(r'/content/drive/MyDrive/Parkdataset/testing/healthy/V03HE1.png',target_size=(299,299))

In [ ]:
x=image.img_to_array(img)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x.shape

(299, 299, 3)

In [ ]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 299, 299, 3)

In [ ]:
model.predict(img_data)

1/1 [==============================] - 2s 2s/step


array([[0.000000e+00, 1.000000e+00, 8.107254e-13]], dtype=float32)

In [ ]:
output=np.argmax(model.predict(img_data), axis=1)

1/1 [==============================] - 0s 250ms/step


In [ ]:
index=['Error', 'healthy', 'parkinson']
result = str(index[output[0]])
result

'healthy'